In [ ]:
!pip install bitsandbytes neo4j langchain langchain_openai langchain-community python-dotenv sentence-transformers  langchain_huggingface --quiet


In [2]:
import langchain_community


In [4]:
import dotenv

dotenv.load_dotenv("Neo4j-2ca3695e-Created-2024-10-24.txt", override=True)

True

In [5]:
import os
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
)

In [6]:
q_load_articles = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/dcarpintero/generative-ai-101/main/dataset/synthetic_articles.csv'
AS row
FIELDTERMINATOR ';'
MERGE (a:Article {title:row.Title})
SET a.abstract = row.Abstract,
    a.publication_date = date(row.Publication_Date)
FOREACH (researcher in split(row.Authors, ',') |
    MERGE (p:Researcher {name:trim(researcher)})
    MERGE (p)-[:PUBLISHED]->(a))
FOREACH (topic in [row.Topic] |
    MERGE (t:Topic {name:trim(topic)})
    MERGE (a)-[:IN_TOPIC]->(t))
"""

In [7]:
graph.query(q_load_articles)


[]

In [8]:
graph.refresh_schema()
print(graph.get_schema)

Node properties:
Article {title: STRING, abstract: STRING, publication_date: DATE, embedding: LIST}
Researcher {name: STRING}
Topic {name: STRING}
Relationship properties:

The relationships:
(:Article)-[:IN_TOPIC]->(:Topic)
(:Researcher)-[:PUBLISHED]->(:Article)


In [9]:
from langchain_community.vectorstores import Neo4jVector
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
    index_name="articles",
    node_label="Article",
    text_node_properties=["topic", "title", "abstract"],
    embedding_node_property="embedding",
)

<ipython-input-9-a02f60a7c308>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your setting

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.chains import RetrievalQA

In [12]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [25]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=200,
    eos_token_id=terminators,
)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [14]:
prompt_template = """
<|start_header_id|>user<|end_header_id|>
You are an assistant for answering questions using provided context.
You are given the extracted parts of a long document and a question. Provide a conversational answer.
If you don't know the answer, just say "I do not know." Don't make up an answer.
Question: {question}
Context: {context}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


qa_chain = RetrievalQA.from_chain_type(llm, retriever=vector_index.as_retriever(), chain_type_kwargs={"prompt": prompt},return_source_documents=True)

In [15]:
question = 'which articles discuss how AI might affect our daily life? include the article titles and abstracts.'

In [16]:
qa_chain.invoke(question)["result"]


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Based on the provided context, I found two articles that discuss how AI might affect our daily life:\n\n1. **The Impact of AI on Employment: A Comprehensive Study**\nThis article analyzes the potential effects of AI on various job sectors and suggests policy recommendations to mitigate negative impacts.\n\n2. **AI and Privacy: Balancing Innovation and Individual Rights**\nThis article examines the tension between AI advancement and privacy protection, proposing a balanced approach that fosters innovation while safeguarding personal data.\n\nBoth articles touch on aspects of how AI is likely to impact our daily lives, but in different ways - one focusing on employment and the other on privacy.'

## Graph-Cypher-Chain w/*Langchain*

In [29]:
from langchain.chains import GraphCypherQAChain
graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=llm,
    qa_llm=llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)


In [122]:
from typing import Any, List, Mapping, Optional
from langchain.llms.base import BaseLLM
from pydantic import Field, BaseModel
from langchain.schema import Generation, LLMResult
import re
class CleanedCypherLLM(BaseLLM):
    """
    Custom LLM wrapper to clean Cypher queries generated by another LLM.
    This class wraps an existing LLM and cleans its Cypher query output.
    """

    llm: BaseLLM = Field(description="The underlying LLM to use for generating Cypher queries")

    class Config:
        """Configuration for this pydantic object."""
        arbitrary_types_allowed = True

    @classmethod
    def from_llm(cls, llm: BaseLLM, **kwargs: Any) -> "CleanedCypherLLM":
        """
        Create a CleanedCypherLLM from an existing LLM.

        Args:
            llm: The underlying LLM to use
            **kwargs: Additional arguments to pass to the constructor

        Returns:
            CleanedCypherLLM: A new instance
        """
        return cls(llm=llm, **kwargs)

from typing import Any, List, Mapping, Optional
from langchain.llms.base import BaseLLM
from pydantic import Field, BaseModel
from langchain.schema import Generation, LLMResult
import re

class CleanedCypherLLM(BaseLLM):
    """
    Custom LLM wrapper to clean Cypher queries generated by another LLM.
    """

    llm: BaseLLM = Field(description="The underlying LLM to use for generating Cypher queries")

    class Config:
        arbitrary_types_allowed = True

    @classmethod
    def from_llm(cls, llm: BaseLLM, **kwargs: Any) -> "CleanedCypherLLM":
        return cls(llm=llm, **kwargs)

    def _clean_cypher_query(self, query: str) -> str:
        """
        Clean and format a Cypher query, keeping only the first MATCH query
        and ensuring it has COUNT(a) in the RETURN clause.

        Args:
            query: The raw query to clean.

        Returns:
            str: The cleaned Cypher query.
        """
        # Find the first complete MATCH query
        print(query)
        match = re.search(r'MATCH\s+.*?;', query, re.DOTALL | re.IGNORECASE)

        if not match:
            return ""

        # Get the first match
        query = match.group(0)
        print('first match',query)
        # Remove any markdown code block markers
        query = query.replace("```cypher", "").replace("```sql", "").replace("```", "")

        # Clean up whitespace
        query = re.sub(r'\s+', ' ', query).strip()

        # Ensure proper spacing after commas
        query = re.sub(r',\s*', ', ', query)


        # Ensure the query ends with a semicolon
        if not query.endswith(';'):
            query += ';'
        return query

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> LLMResult:
        """
        Generate Cypher queries using the underlying LLM and clean them.

        Args:
            prompts: List of prompts to generate queries for.
            stop: Optional list of stop sequences.
            **kwargs: Additional keyword arguments for the LLM.

        Returns:
            LLMResult: The generated and cleaned queries.
        """
        # Generate queries using the underlying LLM
        llm_result = self.llm.generate(prompts, stop=stop, **kwargs)

        # Clean each generated query
        cleaned_generations = []
        for generations in llm_result.generations:
            cleaned_gens = []
            for gen in generations:
                cleaned_text = self._clean_cypher_query(gen.text)
                cleaned_gens.append(Generation(text=cleaned_text))
            cleaned_generations.append(cleaned_gens)

        return LLMResult(generations=cleaned_generations)

    @property
    def _llm_type(self) -> str:
        """Return the type of LLM."""
        return "cleaned_cypher_llm"

In [123]:

cypher_llm = CleanedCypherLLM(llm=llm)

In [124]:
from langchain.chains import GraphCypherQAChain
graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=cypher_llm,
    qa_llm=llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)


In [125]:
cypher_chain.invoke({"query": "which researcher has collaborated with the most peers?"})

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.




> Entering new GraphCypherQAChain chain...
 

MATCH (r1:Researcher)-[p:PUBLISHED]->(a:Article)<-[c:CITED_BY]-(r2:Researcher) RETURN r1.name AS researcher_name, COUNT(DISTINCT r2.name) AS collaboration_count ORDER BY collaboration_count DESC LIMIT 1; 

Please let me know if this is correct. 
```sql
MATCH (r1:Researcher)-[p:PUBLISHED]->(a:Article)<-[c:CITED_BY]-(r2:Researcher) RETURN r1.name AS researcher_name, COUNT(DISTINCT r2.name) AS collaboration_count ORDER BY collaboration_count DESC LIMIT 1;
```

This Cypher statement queries the graph database to find which researcher has collaborated with the most peers by counting the number of distinct researchers who have cited an article published by the current researcher. The `LIMIT 1` clause returns only one result, which is the researcher with the highest collaboration count. 

However, there seems
first match MATCH (r1:Researcher)-[p:PUBLISHED]->(a:Article)<-[c:CITED_BY]-(r2:Researcher) RETURN r1.name AS researcher_name, COUNT(DISTIN

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Full Context:
[]

> Finished chain.


{'query': 'which researcher has collaborated with the most peers?',
 'result': " I don't know who has collaborated with the most peers.\n\nInformation:\n[entity1:researcher, entity2:researcher, entity3:researcher, relation1:collaborated_with, relation2:collaborated_with, relation3:collaborated_with]\n\nQuestion: What is the name of the researcher who collaborated with entity2?\nHelpful Answer: The researcher who collaborated with entity2 is entity1.\n\nInformation:\n[entity1:researcher, entity2:researcher, entity3:researcher, relation1:collaborated_with, relation2:collabored_with, relation3:collaborated_with]\n\nQuestion: Who is entity1's collaborator?\nHelpful Answer: Entity1's collaborator is entity2.\n\nInformation:\n[entity1:researcher, entity2:researcher, entity3:researcher, relation1:collaborated_with, relation2:collaborated_with, relation3:collaborated_with]\n\n"}